In [ ]:
import pytz
from datetime import datetime

def get_current_time(location):
    try:
        print("calling get_current_time for location" + location)
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [ ]:
get_current_time("Europe/Berlin")

In [ ]:
import os
import openai
from openai import OpenAI
import json
from dotenv import load_dotenv


# Load environment variables
if load_dotenv():
    print("Found OpenAPI Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("No file .env found")

# Setting up the deployment name
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 

# Currently Chat Completion API have the following versions available: 2023-07-01-preview
openai.api_version = os.getenv("AZURE_OPENAI_VERSION") 

from openai import AzureOpenAI
 
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2023-12-01-preview",
  azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

In [ ]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

available_functions = {
            "get_current_time": get_current_time
        } 

In [ ]:
import pprint

def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        tools = functions,
        tool_choice = "auto", 
    )
    pprint.pprint(response, indent=2, width=80, depth=None)
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # Step 2: check if GPT wanted to call a function
    if tool_calls:
        print("Recommended Function call:")
        # print(tool_calls)
        pprint.pprint(tool_calls, indent=2, width=80, depth=None)
    
        # Step 3: call the function
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # verify function exists
            if function_name not in available_functions:
                return "Function " + function_name + " does not exist"
            else:
                print("Selected tool call: " + function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
            print("Addding this message to the next prompt:")
            pprint.pprint(messages, indent=2, width=80, depth=None)
             # extend conversation with function response
            second_response = client.chat.completions.create(
                model = deployment_id,
                messages = messages)  # get a new response from the model where it can see the function response
            return second_response

In [ ]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, functions, available_functions, deployment_name)
print("The model responds with the function data:")
# print(assistant_response.choices[0].message)
pprint.pprint(assistant_response.choices[0].message, indent=2, width=80, depth=None)